In [ ]:
# Databricks notebook source
import mlflow
from mlflow.tracking import MlflowClient
import os
import time

print("🚀 Unity Catalog में मॉडल को 'Staging' में ट्रांज़िशन करने की प्रक्रिया शुरू हो रही है।")

# =====================================================
# MLflow Registry Configuration (Unity Catalog)
# =====================================================
try:
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        mlflow.set_registry_uri("databricks-uc")
        print("✅ MLflow रजिस्ट्री URI को यूनिटी कैटलॉग पर सेट किया गया।")
    else:
        print("⚠️ Databricks वातावरण में नहीं चल रहा है। URI सेट करना छोड़ दिया गया है।")
        
    client = MlflowClient()
    print("✅ MLflow क्लाइंट सफलतापूर्वक इनिशियलाइज़ किया गया।")
except Exception as e:
    print(f"❌ MLflow क्लाइंट इनिशियलाइज़ेशन विफल हुआ: {e}")
    exit()

# =====================================================
# 🔍 Auto-detect latest trained model (based on experiment name)
# =====================================================
print("\n🔍 Auto-detecting model name from latest finished experiment...")

try:
    experiments = client.search_experiments()
    finished_exps = []

    # Collect all experiments that have at least one finished run
    for exp in experiments:
        runs = client.search_runs(
            [exp.experiment_id],
            filter_string="attributes.status = 'FINISHED'",
            order_by=["attributes.start_time DESC"],
            max_results=1
        )
        if runs:
            finished_exps.append((exp, runs[0].info.start_time))
    
    if not finished_exps:
        raise Exception("No finished experiments found in MLflow.")

    # Get the experiment with the most recent finished run
    latest_exp = sorted(finished_exps, key=lambda x: x[1], reverse=True)[0][0]
    exp_name = latest_exp.name.lower()
    print(f"📘 Latest Finished Experiment: {latest_exp.name}")

    # 🔁 Map experiment name keywords to registered model names
    model_map = {
        "xgboost": "house_price_xgboost_uc",
        "randomforest": "house_price_rf_uc",
        "lightgbm": "house_price_lightgbm_uc",
        "catboost": "house_price_catboost_uc",
        "gradientboosting": "house_price_gb_uc",
        "linear": "house_price_linear_uc",
        "decisiontree": "house_price_dt_uc",
        "logistic": "house_price_logreg_uc",
        "svm": "house_price_svm_uc"
    }

    detected_model = None
    for key, val in model_map.items():
        if key in exp_name:
            detected_model = val
            print(f"✅ Detected Model Type: {key.upper()}")
            break

    if not detected_model:
        detected_model = "house_price_generic_uc"
        print("⚠️ No keyword matched — using generic model name.")

    UC_CATALOG_NAME = "workspace"
    UC_SCHEMA_NAME = "ml"
    model_name = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{detected_model}"
    print(f"✅ Auto-detected registered model name: {model_name}")

except Exception as e:
    print(f"⚠️ Auto-detection failed: {e}")
    print("⏩ Falling back to default model name...")
    model_name = "workspace.ml.house_price_generic_uc"

# =====================================================
# Transition the detected model to STAGING
# =====================================================
staging_alias = "Staging"

try:
    model_versions = client.search_model_versions(f"name='{model_name}'")
    
    if not model_versions:
        print(f"❌ मॉडल '{model_name}' के कोई संस्करण नहीं मिले। क्या आपने इसे प्रशिक्षित और पंजीकृत किया है?")
    else:
        latest_version = sorted(model_versions, key=lambda mv: int(mv.version), reverse=True)[0]
        version_number = latest_version.version
        print(f"ℹ️ नवीनतम मॉडल संस्करण मिला: v{version_number}")

        # Wait until model status becomes READY
        max_wait_time = 300  # seconds
        elapsed_time = 0
        check_interval = 5
        
        while elapsed_time < max_wait_time:
            current_version = client.get_model_version(model_name, version_number)
            status = current_version.status
            print(f"⏳ मॉडल v{version_number} की वर्तमान स्थिति: '{status}'")

            if status == "READY":
                print(f"✅ मॉडल v{version_number} अब 'READY' स्थिति में है। उपनाम सेट कर रहा हूँ।")
                
                # Set alias = "Staging"
                client.set_registered_model_alias(model_name, staging_alias, version_number)
                print(f"✅ मॉडल संस्करण v{version_number} को सफलतापूर्वक '{staging_alias}' उपनाम पर सेट किया गया है।")
                
                # Optional verification
                try:
                    model_info = client.get_registered_model(model_name)
                    if hasattr(model_info, 'aliases') and staging_alias in model_info.aliases:
                        print(f"🎯 Verification: '{staging_alias}' alias successfully set!")
                    else:
                        print("⚠️ Warning: Alias set हो गया है लेकिन verification में issue है।")
                except Exception as verify_error:
                    print(f"⚠️ Alias verification में समस्या: {verify_error}")
                break

            elif status == "FAILED_REGISTRATION":
                print(f"❌ मॉडल registration failed हो गया है। Status: '{status}'")
                print("❗ कृपया मॉडल को फिर से train और register करें।")
                break

            elif status in ["PENDING_REGISTRATION", "REGISTERING"]:
                print(f"⏳ मॉडल अभी भी '{status}' स्थिति में है। {check_interval} सेकंड प्रतीक्षा कर रहा हूँ...")
                time.sleep(check_interval)
                elapsed_time += check_interval

            else:
                print(f"🔍 अज्ञात स्थिति: '{status}'. {check_interval} सेकंड प्रतीक्षा कर रहा हूँ...")
                time.sleep(check_interval)
                elapsed_time += check_interval
                
        if elapsed_time >= max_wait_time:
            current_version = client.get_model_version(model_name, version_number)
            final_status = current_version.status
            print(f"⏰ प्रतीक्षा समय समाप्त हो गया। मॉडल अभी भी '{final_status}' स्थिति में है।")
            print("❗ कृपया MLflow UI में मॉडल की स्थिति मैन्युअल रूप से जांचें।")
            print("💡 सुझाव: मॉडल registration में कोई समस्या हो सकती है।")

except Exception as e:
    print(f"❌ मॉडल को '{staging_alias}' में ट्रांज़िशन करने में त्रुटि: {e}")
    print("\n🔍 डिबगिंग जानकारी:")
    print(f"1. मॉडल नाम: {model_name}")
    print(f"2. Staging alias: {staging_alias}")
    print("3. कृपया सुनिश्चित करें कि:")
    print("   - मॉडल सही नाम से registered है")
    print("   - आपके पास model access permissions हैं") 
    print("   - Unity Catalog properly configured है")
